# cvxopt - Quadratic Programming solver

In this notebook we will explore the use of the Quadratic Programming solver **`cvxopt`** to compute a support vector machine SVM.

**Installation guide** (via **[Stackoverflow](https://stackoverflow.com/a/21885770/4019402)**)

To install `cvxopt` in Anaconda, open the Anaconda prompt in administrator mode (in Windows go to Start -> Programs and rightclick on the Anaconda prompt icon to open it in administrator mode), and type:

`conda install -c https://conda.anaconda.org/omnia cvxopt`

If you have Python 3.6, and Anaconda complains about requiring Python 3.5, then you can install Python 3.5 as described **[here](https://conda.io/docs/user-guide/tasks/manage-python.html)**

Afterwards, open the Anaconda Navigator and under "Environments" click on the py35 environment to open Jupyter.

## Import libraries

In [1]:
import sys
print(sys.version)

import cvxopt
import numpy as np
import pandas as pd

3.5.4 |Anaconda custom (64-bit)| (default, Sep 19 2017, 08:15:17) [MSC v.1900 64 bit (AMD64)]


# Ghost problem

The following is a ghost problem devised by @Ilikec. We are given a set of data points $X$ with labels $y$. We can compute the support vector machine for $X$ and $y$.

Note: Ghost problems are problems devised by students that we could discuss openly on the edX discussion forum. This one served to answer questions about support vector machines

In [2]:
X = np.array([[-1.5, 1], [2.5, -3], [3.4, -5], [6.4, -2], [3.4, 1.5]])
y = np.array([1, -1, -1, -1, 1])

#df = pd.DataFrame(np.c_[X, y], columns = ['x1', 'x2', 'y'])
df = pd.DataFrame({'x1':X[:,0], 'x2':X[:,1], 'y':y})
print(df)

    x1   x2  y
0 -1.5  1.0  1
1  2.5 -3.0 -1
2  3.4 -5.0 -1
3  6.4 -2.0 -1
4  3.4  1.5  1


In **`cvxopt`** we have to specify the matrices for the following problem, see also **[here](https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf)** : 

$\min \frac{1}{2} x^T P x + q^T x$

subject to $Gx \preceq h$

$Ax = b$

This looks similar to the problem we had in lecture 14, slide 15:

![lecture14_slide15](quadratic_programming_lecture.png)

The `cvxopt` solver requires us to define the parameters $P, q, G, h, A, b$.

- The $\alpha$ from the lecture corresponds to the $x$ from the solver.
- The matrix $P$ from the solver is the matrix on slide 15. It can be calculated by the [Hadamard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices) $YY \circ XX$, where $YY = y^T y$ and $XX = X^T X$. Note that the factor $\frac{1}{2}$ from the slides is already accounted for in the problem formulation of the solver.
- The vector $q$ is simply a vector of negatives ones.
- Next we have the constraint $\alpha \geq 0$. Since the solver formulates the constraint as  $Gx \preceq h$, we have to translate our $\alpha$ constraint as follows: $-\alpha \leq 0$ corresponds to $Gx \preceq h$, where $G$ is simply the negative identity matrix, and $h$ the zero vector.
- The linear constraint $y^T \alpha = 0$ from the lecture corresponds to the matrix equation $Ax = b$ from the solver, where $A$ is the vector $y$ and $b$ is the number zero (scalar). 

In [3]:
N = X.shape[0]
YY = np.dot(y.reshape(N,1),y.reshape(1,N))
XX = np.dot(X, X.T)


# http://cvxopt.org/examples/tutorial/qp.html
# tell program that the alpha_i from the lecture are nonnegative

G = cvxopt.matrix(-np.identity(N), tc='d')
h = cvxopt.matrix(np.zeros(N), tc='d')

P = cvxopt.matrix(YY * XX, tc='d')
q = cvxopt.matrix(-np.ones(N), tc='d')
A = cvxopt.matrix(y, tc='d').T
b = cvxopt.matrix(np.zeros(1), tc='d')

from cvxopt import solvers
sol = solvers.qp(P, q, G, h, A, b)
print("alpha =")
print(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -4.8949e-01 -8.0087e-01  9e+00  3e+00  1e+00
 1:  8.3960e-02 -3.8656e-01  6e-01  5e-02  2e-02
 2: -5.8692e-02 -1.4982e-01  9e-02  2e-03  1e-03
 3: -1.0780e-01 -1.5626e-01  5e-02  6e-04  3e-04
 4: -1.1571e-01 -1.2284e-01  7e-03  8e-05  4e-05
 5: -1.1690e-01 -1.1707e-01  2e-04  1e-17  4e-16
 6: -1.1695e-01 -1.1695e-01  2e-06  3e-17  4e-16
 7: -1.1695e-01 -1.1695e-01  2e-08  2e-17  3e-16
Optimal solution found.
alpha =
[ 9.43e-09]
[ 5.92e-02]
[ 1.63e-09]
[ 5.78e-02]
[ 1.17e-01]



From $\alpha$ we can compute $\mathbf{w} = \sum_{i=1}^N \alpha_i y_i \mathbf{x_i}$ (see slide 14 of lecture 14).

In [4]:
alpha = np.array(sol['x'])
n = alpha.shape[0]

w = np.zeros(X.shape[1])
for i in range(n):
    w = w + y[i]*alpha[i] * X[i]

print("w = ", w)

w =  [-0.1201201  0.4684685]


The solver returns the vector $\alpha$ from the lecture, which are the Lagrange multipliers. Most of the entries in $\alpha$ are zero. The nonzero entries indicate a support vector, i.e. points $x_n$ that are on the margin and that satisfy

$y_n (w^T x_n + b) = 1$

Let's print our support vectors.

In [5]:
# nonzero entries in alpha
indices = (alpha > 10**(-6)).nonzero()[0]    # count boolean true values

# support vectors
support_vectors = X[indices]
print("Support vectors:\n", support_vectors)

num_support_vectors = sum(alpha > 10**(-6))
print("\nNumber of support vectors: ", num_support_vectors)

Support vectors:
 [[ 2.5 -3. ]
 [ 6.4 -2. ]
 [ 3.4  1.5]]

Number of support vectors:  [3]


We can also compute the intercept term $b$ from the equation

$y_n (w^T x_n + b) = 1$

where $x_n$ is a support vector, i.e. a point on the margin.

$y_n (w^T x_n + b) = 1$

$\Rightarrow b = 1/y_n - w^T x_n$


In [6]:
for i in indices:
    b_intercept = 1 / y[i] - np.dot(w, X[i])
    print("b = ", b_intercept)

b =  0.70570575504
b =  0.705705649232
b =  0.705705593249


## Python class

Let's write a Python class:

In [7]:
class SVM_cvxopt:
    
    def __init__(self):
        self.num_support_vectors_ = None
        self.w_ = None
        self.intercept_ = None
        self.support_vectors_ = None
        self.alpha_ = None
        
    
    def fit(self, X, y):
        '''
        - Takes data X and labels y
        - Calculates weight vector
        '''
        N = X.shape[0]
        YY = np.dot(y.reshape(N,1),y.reshape(1,N))
        XX = np.dot(X, X.T)

        # http://cvxopt.org/examples/tutorial/qp.html
        # tell program that the alpha_i (Lagrange multipliers) are nonnegative
        G = cvxopt.matrix(-np.identity(N), tc='d')
        h = cvxopt.matrix(np.zeros(N), tc='d')

        P = cvxopt.matrix(YY * XX, tc='d')
        q = cvxopt.matrix(-np.ones(N), tc='d')
        A = cvxopt.matrix(y, tc='d').T
        b = cvxopt.matrix(np.zeros(1), tc='d')

        from cvxopt import solvers
        # https://stackoverflow.com/questions/26416626/how-to-silent-cvxopt-solver-python
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        
        # alpha vector (see lecture, Lagrange multipliers)
        # https://stackoverflow.com/questions/3337301/numpy-matrix-to-array
        self.alpha_ = np.asarray(sol['x']).reshape(-1)
        n = self.alpha_.shape[0]
        
        # vector w
        self.w_ = np.zeros(X.shape[1])
        for i in range(n):
            self.w_ = self.w_ + y[i] * self.alpha_[i] * X[i]
        
        # threshold to determine which of the entries in alpha are nonzero
        threshold = 10**(-4)
        
        # number of support vectors is the number of non-zero entries in alpha
        self.num_support_vectors_ = sum(self.alpha_ > threshold)
        
        # support vectors
        self.indices_ = (self.alpha_ > threshold).nonzero()[0]    # count boolean true values
        
        # https://stackoverflow.com/questions/3337301/numpy-matrix-to-array
        self.support_vectors_ = np.asarray(X[self.indices_]).reshape(-1)
        
        # intercept
        self.intercept_ = 1/y[self.indices_[0]] - np.dot(self.w_, X[self.indices_[0]])
        

**Note**:

The threshold for counting a support vector was here set to $10^{-4}$ in the class above. If you set this value too low, then you will count too many points as support vectors, e.g. if you have an entry of $10^{-6}$ in the $\alpha$ vector, do you count it as support vector or not?

If you set your threshold to $10^{-8}$, then you count that point as a support vector, whereas for a threshold of $10^{-4}$ you don't count it as support vector.

Try it yourself, change the threshold and compare the number of support vectors you get with the scikit learn (libsvm) implementation.

## Instantiate an object

Here, we just calculate the values again using the class from above.

In [8]:
clf = SVM_cvxopt()
clf.fit(X, y)

print("\nnumber of support vectors: ", clf.num_support_vectors_)
print("\nvector alpha = ", clf.alpha_)
print("\nvector w = ", clf.w_)
print("\nintercept = ", clf.intercept_)
print("\nsupport vectors = ", clf.support_vectors_)


number of support vectors:  3

vector alpha =  [  9.43285070e-09   5.91582758e-02   1.62903634e-09   5.77875011e-02
   1.16945769e-01]

vector w =  [-0.1201201  0.4684685]

intercept =  0.70570575504

support vectors =  [ 2.5 -3.   6.4 -2.   3.4  1.5]


## Comparison with the svm solver from sklearn (which uses libsvm)

In [9]:
from sklearn import svm

clf2 = svm.SVC(C = np.inf, kernel = 'linear')
clf2.fit(X, y)

# get support vectors
print("suport vectors: \n", clf2.support_vectors_)

print("\nnumber of support vectors in each class: ", clf2.n_support_ )

print("\nw = ", clf2.coef_)
print("\nintercept = ", clf2.intercept_)

suport vectors: 
 [[ 2.5 -3. ]
 [ 6.4 -2. ]
 [ 3.4  1.5]]

number of support vectors in each class:  [2 1]

w =  [[-0.12008858  0.46834539]]

intercept =  [ 0.70543276]
